<a href="https://colab.research.google.com/github/Ting-Wei-Chang626/Primary_AI/blob/main/demo_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Animal sound classification**

Load neccessary module

In [ ]:
import pandas as pd
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os

Transform audio to images

In [ ]:
def get_melspectrogram_db(file_path, sr=None, n_fft=2048, hop_length=512, n_mels=128, fmin=20, fmax=8300, top_db=80):
    wav,sr = librosa.load(file_path,sr=sr)
    if wav.shape[0]<5*sr:
        wav=np.pad(wav,int(np.ceil((5*sr-wav.shape[0])/2)),mode='reflect')
    else:
        wav=wav[:5*sr]
    spec=librosa.feature.melspectrogram(y=wav, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels,fmin=fmin,fmax=fmax)
    spec_db=librosa.power_to_db(spec,top_db=top_db)

    return spec_db


def spec_to_image(spec, eps=1e-6):
    mean = spec.mean()
    std = spec.std()
    spec_norm = (spec - mean) / (std + eps)
    spec_min, spec_max = spec_norm.min(), spec_norm.max()
    spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
    spec_scaled = spec_scaled.astype(np.uint8)

    return spec_scaled

mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Create ESC50 data format

In [ ]:
class ESC50Data(Dataset):
    def __init__(self, base, df, in_col, out_col):
        self.df = df
        self.data = []
        self.labels = []
        self.c2i={}
        self.i2c={}
        self.categories = sorted(df[out_col].unique())
        for i, category in enumerate(self.categories):
            self.c2i[category]=i
            self.i2c[i]=category

        for ind in tqdm(range(len(df))):
            row = df.iloc[ind]
            file_path = os.path.join(base,row[in_col])
            self.data.append(spec_to_image(get_melspectrogram_db(file_path))[np.newaxis,...])
            self.labels.append(self.c2i[row['category']])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

split dataset into training and validation

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NTUCE_AI_worshop_0629/ESC-50/meta/esc50_animal.csv')
df.head()

train = df[df['fold']!=5]
valid = df[df['fold']==5]

train_data = ESC50Data('/content/drive/MyDrive/NTUCE_AI_worshop_0629/ESC-50/audio', train, 'filename', 'category')
valid_data = ESC50Data('/content/drive/MyDrive/NTUCE_AI_worshop_0629/ESC-50/audio', valid, 'filename', 'category')
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=4, shuffle=True)


100%|██████████| 80/80 [00:02<00:00, 35.15it/s]


In [ ]:
import pickle
with open('indtocat.pkl','wb') as f:
  pickle.dump(train_data.i2c, f)

Create the Convolutional Neural Network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

class ESC50Model(nn.Module):
    def __init__(self, input_shape, num_cats=10):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size = 3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size = 3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        self.conv7 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn7 = nn.BatchNorm2d(256)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.bn8 = nn.BatchNorm2d(256)
        self.dense1 = nn.Linear(256*(((input_shape[1]//2)//2)//2)*(((input_shape[2]//2)//2)//2),500)
        self.dropout = nn.Dropout(0.5)
        self.dense2 = nn.Linear(500, num_cats)
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv5(x)
        x = F.relu(self.bn5(x))
        x = self.conv6(x)
        x = F.relu(self.bn6(x))
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv7(x)
        x = F.relu(self.bn7(x))
        x = self.conv8(x)
        x = F.relu(self.bn8(x))
        x = x.view(x.size(0),-1)
        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = self.dense2(x)
        return x

In [ ]:
if torch.cuda.is_available():
    device=torch.device('cuda:0')
else:
    device=torch.device('cpu')

model = ESC50Model(input_shape=(1,128,431), num_cats=10).to(device)
# model.apply(weights_init)


In [ ]:
learning_rate = 2e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
epochs = 60
loss_fn = nn.CrossEntropyLoss()
train_losses=[]
valid_losses=[]

Declare training process

In [ ]:
def setlr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return optimizer


def lr_decay(optimizer, epoch):
    if epoch%10==0:
        new_lr = learning_rate / (10**(epoch//10))
        optimizer = setlr(optimizer, new_lr)
        print(f'Changed learning rate to {new_lr}')
    return optimizer

def train(model, loss_fn, train_loader, valid_loader, epochs, optimizer, train_losses, valid_losses, change_lr=None):
    accuracy_best = 0
    for epoch in tqdm(range(1,epochs+1)):
        model.train()
        batch_losses=[]

        if change_lr:
            optimizer = change_lr(optimizer, epoch)

        for i, data in enumerate(train_loader):
            x, y = data
            # optimizer.zero_grad()
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            loss.backward()
            batch_losses.append(loss.item())

            if i%5==0:
            # if True:
                optimizer.step()        # update parameters of net
                optimizer.zero_grad()

        train_losses.append(batch_losses)
        print(f'Epoch - {epoch} Train-Loss : {np.mean(train_losses[-1])}')
        model.eval()
        batch_losses=[]
        trace_y = []
        trace_yhat = []

        # Validation
        for i, data in enumerate(valid_loader):
            x, y = data
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            trace_y.append(y.cpu().detach().numpy())
            trace_yhat.append(y_hat.cpu().detach().numpy())
            batch_losses.append(loss.item())

        valid_losses.append(batch_losses)
        trace_y = np.concatenate(trace_y)
        trace_yhat = np.concatenate(trace_yhat)
        accuracy = np.mean(trace_yhat.argmax(axis=1)==trace_y)
        print(f'Epoch - {epoch} Valid-Loss : {np.mean(valid_losses[-1])} Valid-Accuracy : {accuracy}')
        if accuracy > accuracy_best:
            accuracy_best = accuracy
            torch.save(model, "/content/drive/MyDrive/NTUCE_AI_worshop_0629/demo_best.pth")
    return model

Start training

In [ ]:
model_ = train(model, loss_fn, train_loader, valid_loader, epochs, optimizer, train_losses, valid_losses, lr_decay)

torch.save(model_, "/content/drive/MyDrive/NTUCE_AI_worshop_0629/demo_last.pth")

  0%|          | 0/60 [00:00<?, ?it/s]

Epoch - 1 Train-Loss : 8.611197458207608
Epoch - 1 Valid-Loss : 7.08762629032135 Valid-Accuracy : 0.2875


  2%|▏         | 1/60 [00:07<07:24,  7.54s/it]

Epoch - 2 Train-Loss : 3.0811526399105786
Epoch - 2 Valid-Loss : 1.6310142785310746 Valid-Accuracy : 0.525


  3%|▎         | 2/60 [00:13<06:16,  6.49s/it]

...

Epoch - 58 Valid-Loss : 1.195100761204958 Valid-Accuracy : 0.6875
Epoch - 59 Train-Loss : 0.07569937620100972


 98%|█████████▊| 59/60 [03:57<00:03,  3.76s/it]

Epoch - 59 Valid-Loss : 1.1719783302920406 Valid-Accuracy : 0.6875
Changed learning rate to 2e-10
Epoch - 60 Train-Loss : 0.06036309548362624


100%|██████████| 60/60 [04:01<00:00,  4.03s/it]

Epoch - 60 Valid-Loss : 1.194910854101181 Valid-Accuracy : 0.675


In [ ]:
test_path = "/content/drive/MyDrive/NTUCE_AI_worshop_0629/1-18757-A-4.wav"
test_data = spec_to_image(get_melspectrogram_db(test_path))


In [ ]:
with open('indtocat.pkl','rb') as f:
  indtocat = pickle.load(f)

In [ ]:
indtocat

{0: 'cat',
 1: 'cow',
 2: 'crow',
 3: 'dog',
 4: 'frog',
 5: 'hen',
 6: 'insects',
 7: 'pig',
 8: 'rooster',
 9: 'sheep'}

In [ ]:
model_.eval()
spec_t = torch.tensor(test_data).to(device, dtype=torch.float32)
pr = model_.forward(spec_t.reshape(1,1,*spec_t.shape))
print(pr)

ind = pr.argmax(dim=1).cpu().detach().numpy().ravel()[0]
print(indtocat[ind])

tensor([[ -1.9977,  -7.4264,   2.2421,   3.6151,  14.4480,   1.8496, -10.2308,
           1.9156,   0.1999,  -6.0684]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
frog


Utilize the prtrained torchvision model

In [ ]:
from torchvision.models import resnet34

resnet_model = resnet34(pretrained=True)
resnet_model.fc = nn.Linear(512,10)
resnet_model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet_model = resnet_model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
learning_rate = 2e-4
optimizer = optim.Adam(resnet_model.parameters(), lr=learning_rate)
epochs = 60
loss_fn = nn.CrossEntropyLoss()
resnet_train_losses=[]
resnet_valid_losses=[]

model_res = train(resnet_model, loss_fn, train_loader, valid_loader, epochs, optimizer, resnet_train_losses, resnet_valid_losses, lr_decay)
torch.save(model_res, "/content/drive/MyDrive/NTUCE_AI_worshop_0629/demo_resnet.pth")

  0%|          | 0/60 [00:00<?, ?it/s]

Epoch - 1 Train-Loss : 1.5688763864338398
Epoch - 1 Valid-Loss : 0.8573809742927552 Valid-Accuracy : 0.7375


  2%|▏         | 1/60 [00:03<03:45,  3.83s/it]

Epoch - 2 Train-Loss : 0.7663656551390886
Epoch - 2 Valid-Loss : 0.526629748288542 Valid-Accuracy : 0.8375


  3%|▎         | 2/60 [00:07<03:37,  3.75s/it]

...

Epoch - 58 Valid-Loss : 0.304550259798998 Valid-Accuracy : 0.9125
Epoch - 59 Train-Loss : 0.10026359476323705


 98%|█████████▊| 59/60 [03:03<00:03,  3.05s/it]

Epoch - 59 Valid-Loss : 0.2731340953207109 Valid-Accuracy : 0.9
Changed learning rate to 2e-10
Epoch - 60 Train-Loss : 0.09968608622439205


100%|██████████| 60/60 [03:06<00:00,  3.12s/it]

Epoch - 60 Valid-Loss : 0.27116550093051045 Valid-Accuracy : 0.925


In [ ]:
model_res.eval()
pr = model_res.forward(spec_t.reshape(1,1,*spec_t.shape))
print(pr)

ind = pr.argmax(dim=1).cpu().detach().numpy().ravel()[0]
print(indtocat[ind])

tensor([[-1.9101, -2.7312, -0.9272, -1.6028, 10.2817,  0.0989, -2.5776, -1.2754,
         -3.8266, -2.2465]], device='cuda:0', grad_fn=<AddmmBackward0>)
frog


Check paramter number for both models

In [ ]:
# for the custom model
total_params = sum(
	param.numel() for param in model_.parameters()
)
trainable_params = sum(
	p.numel() for p in model_.parameters() if p.requires_grad
)
print("-----Our model------")
print("Total parameter number: ", total_params)
print("Trainable parameter number: ", trainable_params)

# for the resnet model
total_params = sum(
	param.numel() for param in model_res.parameters()
)
trainable_params = sum(
	p.numel() for p in model_res.parameters() if p.requires_grad
)

print("-----Resnet model------")
print("Total parameter number: ", total_params)
print("Trainable parameter number: ", trainable_params)



-----Our model------
Total parameter number:  109723110
Trainable parameter number:  109723110
-----Resnet model------
Total parameter number:  21283530
Trainable parameter number:  21283530
